In [5]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [6]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [7]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [8]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [1]:
# tools.py (Simulated External Tools for Function Calling)

import json

# --- National Park Planner Tools ---

def get_park_info(park_name: str) -> str:
    """Returns key information about a national park, including popular spots and size."""
    # In a real app, this would query a database or API (e.g., NPS API)
    if "Yosemite" in park_name:
        return json.dumps({
            "name": "Yosemite National Park",
            "size_acres": "760,000",
            "popular_sights": ["El Capitan", "Half Dome", "Yosemite Falls", "Mariposa Grove"],
            "recommended_activity": "Hiking the Mist Trail",
            "best_time_of_year": "Late Spring to Early Fall"
        })
    return f"Could not find detailed information for {park_name}."

# --- Accommodation Finder Tool ---

def find_nearby_lodging(park_name: str, number_of_options: int = 3) -> str:
    """Searches for and returns a list of lodging options near the given national park."""
    # In a real app, this would query an API like Booking.com, Airbnb, etc.
    if "Yosemite" in park_name:
        return json.dumps([
            {"name": "The Majestic Yosemite Hotel", "distance": "Inside Park", "type": "Luxury Hotel"},
            {"name": "Yosemite Valley Lodge", "distance": "Inside Park", "type": "Mid-range Hotel"},
            {"name": "Rush Creek Lodge at Yosemite", "distance": "5 miles outside park", "type": "Resort"},
        ])
    return "No nearby lodging found."

# --- Tesla Chargers Agent Tool ---

def find_tesla_superchargers(park_name: str) -> str:
    """Locates the nearest Tesla Supercharger or destination charger stations to the national park."""
    # In a real app, this would query a dedicated charging network API
    if "Yosemite" in park_name:
        return json.dumps([
            {"location": "Mariposa, CA", "distance_miles": "45", "type": "Supercharger", "stalls": 12},
            {"location": "Groveland, CA (Hotel)", "distance_miles": "25", "type": "Destination Charger"},
        ])
    return "No nearby Tesla chargers found."

# Dictionary mapping tool names to their functions
TOOL_MAP = {
    "get_park_info": get_park_info,
    "find_nearby_lodging": find_nearby_lodging,
    "find_tesla_superchargers": find_tesla_superchargers
}

# List of all tools to expose to the model
ALL_TOOLS = [
    get_park_info,
    find_nearby_lodging,
    find_tesla_superchargers
]

In [11]:

async def run_agent(task_prompt: str, tools: list[callable]) -> str:
    """
    A single asynchronous agent function that uses Gemini to call tools and generate a response.
    """
    print(f"Starting agent: {task_prompt.splitlines()[0]}...")
    
    # 1. Initial Prompt
    root_agent = Agent(
    name="root_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
        ),
    tools=tools,
    )
        
    # 2. Check for Function Calls and Execute
    while response.function_calls:
        tool_calls = response.function_calls
        tool_outputs = []
        
        for call in tool_calls:
            function_name = call.name
            args = dict(call.args)
            
            if function_name in TOOL_MAP:
                # Execute the actual Python function
                result = TOOL_MAP[function_name](**args)
                tool_outputs.append(types.Part.from_function_response(
                    name=function_name, 
                    response={'result': result}
                ))
            else:
                # Handle unknown function
                tool_outputs.append(types.Part.from_function_response(
                    name=function_name, 
                    response={'result': f"Tool {function_name} not found."}
                ))

        # 3. Send Tool Outputs back to the model for final synthesis
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=[task_prompt] + tool_outputs, # Pass the original prompt + tool results
            config=types.GenerateContentConfig(
                tools=tools
            )
        )
        
    print(f"Agent finished: {task_prompt.splitlines()[0]}.")
    return response.text.strip()


async def trip_planner_orchestrator(park_name: str, num_days: int):
    """
    Main function to run all three agents in parallel.
    """
    
    # --- Agent Task Prompts ---
    
    planner_prompt = f"""
    You are the National Park Trip Planner Agent. 
    Task: Create a detailed, day-by-day itinerary for a {num_days}-day trip to {park_name}. 
    Use the 'get_park_info' tool to gather initial data, then structure the trip logically (e.g., Day 1: Arrive and short hike, Day 2: Main attractions, Day {num_days}: Final activity and departure).
    """

    lodging_prompt = f"""
    You are the Accommodation Finder Agent. 
    Task: Find 3 highly-rated lodging options for a trip to {park_name}. 
    Use the 'find_nearby_lodging' tool and present the options clearly with their distance and type.
    """

    charger_prompt = f"""
    You are the Tesla Chargers Agent. 
    Task: Find the nearest charging options for a Tesla driver visiting {park_name}. 
    Use the 'find_tesla_superchargers' tool and list the location, distance, and type of charger (Supercharger vs. Destination Charger).
    """
    
    # --- Run Agents in Parallel ---
    
    tasks = [
        run_agent(planner_prompt, tools=[get_park_info]),
        run_agent(lodging_prompt, tools=[find_nearby_lodging]),
        run_agent(charger_prompt, tools=[find_tesla_superchargers]),
    ]
    
    # Use asyncio.gather to run all tasks concurrently
    planner_result, lodging_result, charger_result = await asyncio.gather(*tasks)
    
    # --- Final Trip Compilation ---
    
    final_report = f"""
    # 🌲 Your {park_name} Trip Planner Report ({num_days} Days)

    ---

    ## 🗓️ Day-by-Day Itinerary
    {planner_result}

    ---

    ## 🏡 Accommodation Suggestions
    {lodging_result}

    ---

    ## ⚡ Tesla Charging Options
    {charger_result}
    """
    
    print("\n" + "="*80)
    print(final_report)
    print("="*80)
    
# --- Execution Example ---

if __name__ == "__main__":
    PARK = "Yosemite National Park"
    DAYS = 3
    
    print(f"*** Starting Trip Planning for {PARK} over {DAYS} days ***")
    
    # Run the main asynchronous function
    runner = InMemoryRunner(agent=root_agent)
    response = await runner.run_debug(
    "Plan for Yosemite National Park for 3 days"
)

*** Starting Trip Planning for Yosemite National Park over 3 days ***


/usr/lib/python3.11/codeop.py:125: RuntimeWarning: coroutine 'trip_planner_orchestrator' was never awaited
  codeob = compile(source, filename, symbol, flags, True)


NameError: name 'root_agent' is not defined